In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Installing pyspark
!pip install pyspark

  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [3]:
# Installing findspark
!pip install findspark

In [4]:
# Installing java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [5]:
# Installing spark
!wget -nc https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz

File ‘spark-3.3.1-bin-hadoop2.tgz’ already there; not retrieving.



In [6]:
import findspark
findspark.init()

In [7]:
import pyspark.sql.functions as pyf

In [8]:
# Creating a spark session
from pyspark.sql import DataFrame, SparkSession
spark = SparkSession.builder.appName("House Price Prediction").getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/08 14:21:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/08 14:21:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/08 14:21:41 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [9]:
# Specifying the file path
file_location = "data/test.csv"
file_type = "csv"
# CSV options
infer_schema = True
first_row_is_header = True
delimiter = ","

In [10]:
# The applied options are for CSV files. For other file types, these will be ignored.
DF = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [11]:
# Reviewing the dataset
DF.show(10, False)

23/02/08 14:21:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+----+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+--

In [12]:
# Printing the info/schema 
DF.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

In [13]:
# Checking for null values
from pyspark.sql.functions import isnull, when, count, col
DF.select([count(when(isnull(c), c)).alias(c) for c in DF.columns]).show()

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgType

In [14]:
# Confirming that there are null values in the dataset
DF.distinct().show()

+----+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
|  Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgTy

In [15]:
# Replacing NA to null values for imputation
def fixing_null_values(DF, col_name):
  if col_name != 'MasVnrType' and col_name != 'Electrical':
    new_DF = DF.withColumn(col_name, when((col(col_name)=='NA'), None).otherwise(col(col_name)).cast("float"))
  else:
    new_DF = DF.withColumn(col_name, when((col(col_name)=='NA'), None).otherwise(col(col_name)))
  return new_DF

DF2 = fixing_null_values(DF, 'LotFrontage')
DF3 = fixing_null_values(DF2, 'MasVnrType')
DF4 = fixing_null_values(DF3, 'MasVnrArea')
DF5 = fixing_null_values(DF4, 'Electrical')
DF6 = fixing_null_values(DF5, 'GarageYrBlt')
DF6.show()

+----+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
|  Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgTy

In [16]:
# Confirming that there are null values in the dataset
DF6.distinct().show()

+----+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
|  Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgTy

In [17]:
from pyspark.sql.functions import isnull, when, count, col
DF6.select([count(when(isnull(c), c)).alias(c) for c in DF6.columns]).show()

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgType

In [18]:
# Null value imputation
from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols=['LotFrontage', 'MasVnrArea',  'GarageYrBlt'], outputCols=["{}_imputed".format(c) for c in ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']]).setStrategy("median")

In [19]:
imputer.fit(DF6).transform(DF6).show()

+----+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+-------------------+------------------+-------------------+
|  Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|Lo

In [20]:
# Replacing the null values in categorical columns
DF7 = DF6.na.fill("None",["MasVnrType"]).na.fill("SBrkr",["Electrical"])

In [21]:
# Dropping these columns post null value imputation
DF8 = DF7.drop('LotFrontage', 'MasVnrArea',  'GarageYrBlt')

In [22]:
DF8.show()

+----+----------+--------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
|  Id|MSSubClass|MSZoning|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgType|HouseStyle|OverallQual|OverallCond|YearBuilt

In [23]:
# Checking for null values one last time - no null values found
from pyspark.sql.functions import isnull, when, count, col
DF8.select([count(when(isnull(c), c)).alias(c) for c in DF8.columns]).show()

+---+----------+--------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
| Id|MSSubClass|MSZoning|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgType|HouseStyle|OverallQual|OverallCond|YearBuilt|Y

In [24]:
# Checking for duplicate values - no duplicate values found
DF8.groupBy(DF8.columns).count().filter("count > 1").show()

+---+----------+--------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+-----+
| Id|MSSubClass|MSZoning|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgType|HouseStyle|OverallQual|OverallCond|YearB

In [25]:
DF.columns

['Id',
 'MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF'

In [26]:
DF.summary()

DataFrame[summary: string, Id: string, MSSubClass: string, MSZoning: string, LotFrontage: string, LotArea: string, Street: string, Alley: string, LotShape: string, LandContour: string, Utilities: string, LotConfig: string, LandSlope: string, Neighborhood: string, Condition1: string, Condition2: string, BldgType: string, HouseStyle: string, OverallQual: string, OverallCond: string, YearBuilt: string, YearRemodAdd: string, RoofStyle: string, RoofMatl: string, Exterior1st: string, Exterior2nd: string, MasVnrType: string, MasVnrArea: string, ExterQual: string, ExterCond: string, Foundation: string, BsmtQual: string, BsmtCond: string, BsmtExposure: string, BsmtFinType1: string, BsmtFinSF1: string, BsmtFinType2: string, BsmtFinSF2: string, BsmtUnfSF: string, TotalBsmtSF: string, Heating: string, HeatingQC: string, CentralAir: string, Electrical: string, 1stFlrSF: string, 2ndFlrSF: string, LowQualFinSF: string, GrLivArea: string, BsmtFullBath: string, BsmtHalfBath: string, FullBath: string, H

In [27]:
columnList = [item[0] for item in DF8.dtypes if item[1].startswith('string')]

In [28]:
output_column_list = list(map(lambda x: x+"_index", columnList))

In [29]:
# String indexer
from pyspark.ml.feature import StringIndexer, OneHotEncoder
indexers = StringIndexer(inputCols=columnList, 
                         outputCols=output_column_list)
strindexedDF = indexers.fit(DF8).transform(DF8)
DF9 = strindexedDF.select("*")

In [30]:
DF9.show()

+----+----------+--------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+--------------+------------+-----------+--------------+-----------------+---------------+---------------+---------------+------------------+----------------+----------------+-------------

In [31]:
# Dropping columns post String Indexing
DF10 = DF9.drop(*columnList)

In [32]:
DF10.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- 1stFlrSF: integer (nullable = true)
 |-- 2ndFlrSF: integer (nullable = true)
 |-- LowQualFinSF: integer (nullable = true)
 |-- GrLivArea: integer (nullable = true)
 |-- FullBath: integer (nullable = true)
 |-- HalfBath: integer (nullable = true)
 |-- BedroomAbvGr: integer (nullable = true)
 |-- KitchenAbvGr: integer (nullable = true)
 |-- TotRmsAbvGrd: integer (nullable = true)
 |-- Fireplaces: integer (nullable = true)
 |-- WoodDeckSF: integer (nullable = true)
 |-- OpenPorchSF: integer (nullable = true)
 |-- EnclosedPorch: integer (nullable = true)
 |-- 3SsnPorch: integer (nullable = true)
 |-- ScreenPorch: integer (nullable = true)
 |-- PoolArea: integer (nullable = true)
 |-- MiscVal:

In [33]:
DF10.show()

+----+----------+-------+-----------+-----------+---------+------------+--------+--------+------------+---------+--------+--------+------------+------------+------------+----------+----------+-----------+-------------+---------+-----------+--------+-------+------+------+--------------+------------+-----------+--------------+-----------------+---------------+---------------+---------------+------------------+----------------+----------------+--------------+----------------+---------------+--------------+-----------------+-----------------+----------------+---------------+---------------+----------------+--------------+--------------+------------------+------------------+----------------+------------------+----------------+---------------+-----------------+-------------+---------------+----------------+----------------+------------------+------------------+-----------------+----------------+-----------------+----------------+------------------+----------------+----------------+----------

In [34]:
DF10.columns

['Id',
 'MSSubClass',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'MSZoning_index',
 'Street_index',
 'Alley_index',
 'LotShape_index',
 'LandContour_index',
 'Utilities_index',
 'LotConfig_index',
 'LandSlope_index',
 'Neighborhood_index',
 'Condition1_index',
 'Condition2_index',
 'BldgType_index',
 'HouseStyle_index',
 'RoofStyle_index',
 'RoofMatl_index',
 'Exterior1st_index',
 'Exterior2nd_index',
 'MasVnrType_index',
 'ExterQual_index',
 'ExterCond_index',
 'Foundation_index',
 'BsmtQual_index',
 'BsmtCond_index',
 'BsmtExposure_index',
 'BsmtFinType1_index',
 'BsmtFinSF1_index',
 'BsmtFinType2_index',
 'BsmtFinSF2_index',
 'BsmtUnfSF_index',
 'TotalBsmtSF_index',
 'Heating_inde

In [35]:
input_cols = ['MSSubClass',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [36]:
output_cols = list(map(lambda x: x+"_scaled", input_cols))

In [37]:
# Vectorizing the independent features
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=input_cols,outputCol="Independent_Features")

In [38]:
DF11 = featureassembler.transform(DF10)

In [39]:
DF11.show()

+----+----------+-------+-----------+-----------+---------+------------+--------+--------+------------+---------+--------+--------+------------+------------+------------+----------+----------+-----------+-------------+---------+-----------+--------+-------+------+------+--------------+------------+-----------+--------------+-----------------+---------------+---------------+---------------+------------------+----------------+----------------+--------------+----------------+---------------+--------------+-----------------+-----------------+----------------+---------------+---------------+----------------+--------------+--------------+------------------+------------------+----------------+------------------+----------------+---------------+-----------------+-------------+---------------+----------------+----------------+------------------+------------------+-----------------+----------------+-----------------+----------------+------------------+----------------+----------------+----------

In [40]:
# Import StandardScaler from pyspark.ml.feature package
from pyspark.ml.feature import StandardScaler

# Create the StandardScaler object. It only take feature column (dense vector)
stdscaler = StandardScaler(inputCol="Independent_Features", outputCol="Scaled_Features")

# Fit the StandardScaler object on the output of the dense vector data and transform
stdscaledDF = stdscaler.fit(DF11).transform(DF11)
stdscaledDF.select("*").show()

+----+----------+-------+-----------+-----------+---------+------------+--------+--------+------------+---------+--------+--------+------------+------------+------------+----------+----------+-----------+-------------+---------+-----------+--------+-------+------+------+--------------+------------+-----------+--------------+-----------------+---------------+---------------+---------------+------------------+----------------+----------------+--------------+----------------+---------------+--------------+-----------------+-----------------+----------------+---------------+---------------+----------------+--------------+--------------+------------------+------------------+----------------+------------------+----------------+---------------+-----------------+-------------+---------------+----------------+----------------+------------------+------------------+-----------------+----------------+-----------------+----------------+------------------+----------------+----------------+----------

In [41]:
DF12 = stdscaledDF.drop(*input_cols, "Independent_Features")

In [42]:
DF12.show()

+----+--------------+------------+-----------+--------------+-----------------+---------------+---------------+---------------+------------------+----------------+----------------+--------------+----------------+---------------+--------------+-----------------+-----------------+----------------+---------------+---------------+----------------+--------------+--------------+------------------+------------------+----------------+------------------+----------------+---------------+-----------------+-------------+---------------+----------------+----------------+------------------+------------------+-----------------+----------------+-----------------+----------------+------------------+----------------+----------------+----------------+----------------+----------------+------------+-----------+-----------------+--------------+-------------------+--------------------+
|  Id|MSZoning_index|Street_index|Alley_index|LotShape_index|LandContour_index|Utilities_index|LotConfig_index|LandSlope_ind

In [43]:
DF12.columns

['Id',
 'MSZoning_index',
 'Street_index',
 'Alley_index',
 'LotShape_index',
 'LandContour_index',
 'Utilities_index',
 'LotConfig_index',
 'LandSlope_index',
 'Neighborhood_index',
 'Condition1_index',
 'Condition2_index',
 'BldgType_index',
 'HouseStyle_index',
 'RoofStyle_index',
 'RoofMatl_index',
 'Exterior1st_index',
 'Exterior2nd_index',
 'MasVnrType_index',
 'ExterQual_index',
 'ExterCond_index',
 'Foundation_index',
 'BsmtQual_index',
 'BsmtCond_index',
 'BsmtExposure_index',
 'BsmtFinType1_index',
 'BsmtFinSF1_index',
 'BsmtFinType2_index',
 'BsmtFinSF2_index',
 'BsmtUnfSF_index',
 'TotalBsmtSF_index',
 'Heating_index',
 'HeatingQC_index',
 'CentralAir_index',
 'Electrical_index',
 'BsmtFullBath_index',
 'BsmtHalfBath_index',
 'KitchenQual_index',
 'Functional_index',
 'FireplaceQu_index',
 'GarageType_index',
 'GarageFinish_index',
 'GarageCars_index',
 'GarageArea_index',
 'GarageQual_index',
 'GarageCond_index',
 'PavedDrive_index',
 'PoolQC_index',
 'Fence_index',
 'Misc

In [44]:
# Import VectorAssembler from pyspark.ml.feature package
from pyspark.ml.feature import VectorAssembler
# Create a list of all the variables that you want to create feature vectors
# These features are then further used for training model
features_col = ['MSZoning_index',
 'Street_index',
 'Alley_index',
 'LotShape_index',
 'LandContour_index',
 'Utilities_index',
 'LotConfig_index',
 'LandSlope_index',
 'Neighborhood_index',
 'Condition1_index',
 'Condition2_index',
 'BldgType_index',
 'HouseStyle_index',
 'RoofStyle_index',
 'RoofMatl_index',
 'Exterior1st_index',
 'Exterior2nd_index',
 'MasVnrType_index',
 'ExterQual_index',
 'ExterCond_index',
 'Foundation_index',
 'BsmtQual_index',
 'BsmtCond_index',
 'BsmtExposure_index',
 'BsmtFinType1_index',
 'BsmtFinSF1_index',
 'BsmtFinType2_index',
 'BsmtFinSF2_index',
 'BsmtUnfSF_index',
 'TotalBsmtSF_index',
 'Heating_index',
 'HeatingQC_index',
 'CentralAir_index',
 'Electrical_index',
 'BsmtFullBath_index',
 'BsmtHalfBath_index',
 'KitchenQual_index',
 'Functional_index',
 'FireplaceQu_index',
 'GarageType_index',
 'GarageFinish_index',
 'GarageCars_index',
 'GarageArea_index',
 'GarageQual_index',
 'GarageCond_index',
 'PavedDrive_index',
 'PoolQC_index',
 'Fence_index',
 'MiscFeature_index',
 'SaleType_index',
 'SaleCondition_index',
 'Scaled_Features']
# Create the VectorAssembler object
assembler = VectorAssembler(inputCols=features_col, outputCol="independent_features")
DF13 = assembler.transform(DF12)
final_DF = DF13.select('independent_features')

In [45]:
final_DF.show()

+--------------------+
|independent_features|
+--------------------+
|(76,[0,9,20,24,25...|
|(76,[3,6,13,15,16...|
|(76,[3,8,12,21,25...|
|(76,[3,8,12,17,25...|
|(76,[3,4,8,11,15,...|
|(76,[3,6,8,12,15,...|
|(76,[3,8,15,16,19...|
|(76,[3,8,12,21,24...|
|(76,[8,15,16,21,2...|
|(76,[6,15,16,20,2...|
|(76,[0,3,6,11,15,...|
|(76,[0,8,11,12,15...|
|(76,[0,8,11,12,15...|
|(76,[6,8,11,12,15...|
|(76,[6,8,11,15,16...|
|(76,[3,8,12,17,18...|
|(76,[3,6,8,17,18,...|
|(76,[8,9,13,17,18...|
|(76,[3,8,13,15,16...|
|(76,[4,7,8,13,17,...|
+--------------------+
only showing top 20 rows



pyspark.sql.dataframe.DataFrame

We will apply the saved models to our cleaned test dataset

In [46]:
# Linear Regression
from pyspark.ml.regression import LinearRegressionModel
lr_model=LinearRegressionModel.load("generated_models/linearmodel")

In [47]:
# Coefficients
lr_model.coefficients

DenseVector([-2857.0925, -47968.1021, 1643.3955, 263.7653, -2083.8409, 0.0, -1231.1822, -53.0367, 460.9573, -3305.0283, -8459.24, -728.7072, 709.5898, 1183.653, -15633.982, 719.0596, -1010.4721, 2461.3493, 7510.0157, 281.3312, 1172.016, 5356.962, -960.5859, 3243.0437, -248.691, 516.9691, 2970.491, -870.4007, -983.3592, -1905.0134, 7098.3983, -3180.8034, 28.5964, 813.1712, 3240.1565, 3235.964, -541.4267, -702.6708, 34034.8004, 276.3908, -5753.7204, 1688.7708, -965.5654, -6013.0748, 6466.9888, 18682.8416, 7331.9977, 11375.9013, 2100.3906, 5062.4144, -1099.3225, -2407.1152, 8263.6284, -1797.3451, -3399.4822, -1825.9489, 30425.7122, 2471.9599, -84.5084, 2168.9866, 1830.2452, -3814.1159, -516.5243, 2815.1836, 3568.3722, 3910.1823, 3762.0656, 2126.2477, -1083.2873, 1508.8138, 1421.0493, 3793.6418, -2610.3263, -623.8505, -460.5123, -674.1686])

In [48]:
# Intercepts
lr_model.intercept

20906.40520492312

In [49]:
# Prediction
pred_results_lr=lr_model.transform(final_DF)

In [50]:
pred_results_lr.show()

+--------------------+------------------+
|independent_features|        prediction|
+--------------------+------------------+
|(76,[0,9,20,24,25...| 1489813.835004431|
|(76,[3,6,13,15,16...| 868992.3055432143|
|(76,[3,8,12,21,25...|1641859.6081462593|
|(76,[3,8,12,17,25...|1616720.1217185745|
|(76,[3,4,8,11,15,...|1647978.7248497459|
|(76,[3,6,8,12,15,...| 281354.4967799336|
|(76,[3,8,15,16,19...|1567060.2115053588|
|(76,[3,8,12,21,24...| 960317.9055176594|
|(76,[8,15,16,21,2...|1329133.6725010204|
|(76,[6,15,16,20,2...|1784890.5356217374|
|(76,[0,3,6,11,15,...| 1224558.883760914|
|(76,[0,8,11,12,15...|1308567.4337964135|
|(76,[0,8,11,12,15...|1059170.3589270853|
|(76,[6,8,11,12,15...|1790095.3337479106|
|(76,[6,8,11,15,16...| 914572.9128193792|
|(76,[3,8,12,17,18...| 278382.2140470559|
|(76,[3,6,8,17,18,...| 43891.35004307772|
|(76,[8,9,13,17,18...| 326546.1577776787|
|(76,[3,8,13,15,16...|  856606.212390708|
|(76,[4,7,8,13,17,...|-85442.35858420655|
+--------------------+------------

In [51]:
# Random Forest Regression
from pyspark.ml.regression import RandomForestRegressionModel
rf_model=RandomForestRegressionModel.load("generated_models/rfmodel")

In [52]:
# Prediction
pred_results_rf=rf_model.transform(final_DF)

In [53]:
pred_results_rf.show()

+--------------------+------------------+
|independent_features|        prediction|
+--------------------+------------------+
|(76,[0,9,20,24,25...|147163.17771086574|
|(76,[3,6,13,15,16...|180573.98072608537|
|(76,[3,8,12,21,25...|166245.71174900996|
|(76,[3,8,12,17,25...|160813.31645248504|
|(76,[3,4,8,11,15,...|149449.43789227653|
|(76,[3,6,8,12,15,...|153710.95837052216|
|(76,[3,8,15,16,19...|163606.51895264184|
|(76,[3,8,12,21,24...|146811.02504270457|
|(76,[8,15,16,21,2...|156829.58190747543|
|(76,[6,15,16,20,2...|148453.88508068526|
|(76,[0,3,6,11,15,...|167069.44393840863|
|(76,[0,8,11,12,15...|160739.60794460768|
|(76,[0,8,11,12,15...|141079.08760605444|
|(76,[6,8,11,12,15...|158382.87458250608|
|(76,[6,8,11,15,16...| 141577.4497375889|
|(76,[3,8,12,17,18...|164089.42440293374|
|(76,[3,6,8,17,18,...|160474.29269886136|
|(76,[8,9,13,17,18...|160494.94862969645|
|(76,[3,8,13,15,16...|169661.84269305257|
|(76,[4,7,8,13,17,...|219739.27643824913|
+--------------------+------------

In [54]:
# Gradient Boost Regression
from pyspark.ml.regression import GBTRegressionModel
gbr_model=GBTRegressionModel.load("generated_models/gbrmodel")

In [55]:
pred_results_gb = gbr_model.transform(final_DF)

In [56]:
pred_results_gb.show()

23/02/08 14:22:17 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/02/08 14:22:17 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
+--------------------+------------------+
|independent_features|        prediction|
+--------------------+------------------+
|(76,[0,9,20,24,25...| 58205.14714140962|
|(76,[3,6,13,15,16...|65309.127243216964|
|(76,[3,8,12,21,25...| 75897.97012278036|
|(76,[3,8,12,17,25...| 71260.46515116341|
|(76,[3,4,8,11,15,...| 71429.92537059434|
|(76,[3,6,8,12,15,...|65436.241823255914|
|(76,[3,8,15,16,19...| 64621.42688327808|
|(76,[3,8,12,21,24...| 73199.28151798372|
|(76,[8,15,16,21,2...| 69800.48199147012|
|(76,[6,15,16,20,2...| 58233.65416897122|
|(76,[0,3,6,11,15,...|45331.073998627704|
|(76,[0,8,11,12,15...| 50040.66272347527|
|(76,[0,8,11,12,15...| 33601.06821755017|
|(76,[6,8,11,12,15...| 70443.55525345495|
|(76,[6,8,11,15,16...| 77372.81340516632

In [57]:
# Extracting output csv file for submission
id_column = DF12.select("id")

In [58]:
pred_column = pred_results_rf.select("prediction")

In [59]:
id_pandas = id_column.toPandas()

In [60]:
pred_pandas = pred_column.toPandas()

In [61]:
type(id_pandas)

pandas.core.frame.DataFrame

In [62]:
import pandas as pd
submission = pd.concat([id_pandas,pred_pandas], axis=1)

In [63]:
submission

,id,prediction
0,1461,147163.177711
1,1462,180573.980726
2,1463,166245.711749
3,1464,160813.316452
4,1465,149449.437892
...,...,...
1454,2915,128780.992124
1455,2916,127249.487124
1456,2917,167386.945879
1457,2918,145812.913468


In [64]:
submission.to_csv("data/final_predictions.csv", index=False)